# Building Commit Data Objects from Repository

This notebook demonstrates how to extract commit information from the current repository and build structured `Commit` data objects using our data models. We'll process the 5 most recent commits and display them with enhanced formatting.

In [1]:
# Import required libraries
import json
import sys
from collections import defaultdict
from datetime import datetime
from pathlib import Path

import git
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax
from rich.table import Table
from rich.tree import Tree

from auto_release_note_generation.data_models.commit import Commit
from auto_release_note_generation.data_models.shared import (
    Diff,
    FileModification,
    GitActor,
    GitMetadata,
)
from auto_release_note_generation.data_models.utils import GitSHA

In [2]:
# Add the src directory to the Python path
project_root = Path.cwd().parent

sys.path.insert(0, str(project_root / "src"))

# Import GitPython for repository interaction

# For better display

# Import our data models

console = Console()

In [3]:
# Initialize the repository
repo = git.Repo(project_root)

# Display repository information
info_panel = Panel.fit(
    f"[bold blue]Repository:[/bold blue] {repo.working_dir}\n"
    f"[bold blue]Current branch:[/bold blue] {repo.active_branch.name}\n"
    f"[bold blue]Total commits:[/bold blue] {sum(1 for _ in repo.iter_commits())}",
    title="Repository Information",
    border_style="blue",
)
console.print(info_panel)

╭──────────────── Repository Information ────────────────╮
│ Repository: /home/aj/code/auto-release-note-generation │
│ Current branch: aj/ai-83-jupyter-notebook-support      │
│ Total commits: 59                                      │
╰────────────────────────────────────────────────────────╯

In [4]:
# Get the first 5 commits
commits_to_process = list(repo.iter_commits(max_count=5))

# Display commit overview
table = Table(
    title="Recent Commits Overview", show_header=True, header_style="bold magenta"
)
table.add_column("SHA", style="cyan", width=12)
table.add_column("Author", style="green")
table.add_column("Date", style="yellow")
table.add_column("Summary", style="white")

for commit in commits_to_process:
    table.add_row(
        commit.hexsha[:8],
        f"{commit.author.name}",
        datetime.fromtimestamp(commit.authored_date).strftime("%Y-%m-%d %H:%M"),
        commit.summary[:50] + "..." if len(commit.summary) > 50 else commit.summary,
    )

console.print(table)

                                        Recent Commits Overview                                         
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ SHA          ┃ Author     ┃ Date             ┃ Summary                                               ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ c136f0c9     │ 00arun00   │ 2025-07-05 17:19 │ feat(dev): add Jupyter notebook support with depen... │
│ 2110ee30     │ 00arun00   │ 2025-07-05 17:19 │ feat(dev): add Jupyter notebook support with depen... │
│ c56337bb     │ 00arun00   │ 2025-07-05 16:55 │ refactor(tests): migrate to modular strategy archi... │
│ d75065a7     │ 00arun00   │ 2025-07-05 17:21 │ docs: add comprehensive testing patterns documenta... │
│ c0a82db3     │ dependabot │ 2025-07-06 05:21 │ ci(deps): bump the actions group with 6 updates       │
└──────────────┴────────────┴──────────────────┴───────────────────────────────────────────────────────┘

## Helper Functions

Define all the helper functions needed to build commit objects.

In [5]:
# Helper function to create GitActor from GitPython actor


def create_git_actor(actor: git.Actor, timestamp: int) -> GitActor:
    """Convert GitPython Actor to our GitActor model."""
    return GitActor(
        name=actor.name, email=actor.email, timestamp=datetime.fromtimestamp(timestamp)
    )


# Helper function to extract file modifications
def extract_file_modifications(commit: git.Commit) -> list[FileModification]:
    """Extract file modifications from a commit."""
    modifications = []

    # Get the diff compared to the parent (or empty tree for initial commit)
    if commit.parents:
        diffs = commit.parents[0].diff(commit)
    else:
        # Initial commit - compare against empty tree
        diffs = commit.diff(git.NULL_TREE)

    # Get commit stats for accurate line counts
    stats = commit.stats.files

    for diff_item in diffs:
        # Use change_type as the primary indicator
        change_type = diff_item.change_type

        # Force correct path values based on change type
        if change_type == "A":
            # Added file - force path_before to None
            path_before = None
            path_after = diff_item.b_path
            mod_type = "A"
        elif change_type == "D":
            # Deleted file - force path_after to None
            path_before = diff_item.a_path
            path_after = None
            mod_type = "D"
        elif change_type.startswith("R"):  # R90, R100, etc.
            # Renamed file - use both paths
            path_before = diff_item.a_path
            path_after = diff_item.b_path
            mod_type = "R"
        elif change_type == "M":
            # Modified file - paths should be the same
            path_before = diff_item.a_path
            path_after = diff_item.b_path
            mod_type = "M"
        elif change_type.startswith("C"):  # C90, C100, etc.
            # Copied file
            path_before = diff_item.a_path
            path_after = diff_item.b_path
            mod_type = "C"
        elif change_type == "T":
            # Type change
            path_before = diff_item.a_path
            path_after = diff_item.b_path
            mod_type = "T"
        else:
            # Unknown change type
            path_before = diff_item.a_path
            path_after = diff_item.b_path
            mod_type = "X"

        # Get the file path to look up stats
        file_path = path_after if path_after else path_before

        # Get insertions and deletions from commit stats
        insertions = 0
        deletions = 0
        if file_path and file_path in stats:
            insertions = stats[file_path].get("insertions", 0)
            deletions = stats[file_path].get("deletions", 0)

        mod = FileModification(
            path_before=path_before,
            path_after=path_after,
            modification_type=mod_type,
            insertions=insertions,
            deletions=deletions,
        )
        modifications.append(mod)

    return modifications


# Extract branches containing this commit
def get_branches_containing_commit(repo: git.Repo, commit_sha: str) -> list[str]:
    """Get all branches that contain the given commit."""
    branches = []
    for branch in repo.branches:
        if commit_sha in [c.hexsha for c in repo.iter_commits(branch)]:
            branches.append(branch.name)
    return branches


# Extract tags pointing to this commit
def get_tags_for_commit(repo: git.Repo, commit_sha: str) -> list[str]:
    """Get all tags pointing to the given commit."""
    tags = []
    for tag in repo.tags:
        if tag.commit.hexsha == commit_sha:
            tags.append(tag.name)
    return tags


# Function to build a Commit object from a GitPython commit
def build_commit_object(git_commit: git.Commit, repo: git.Repo) -> Commit:
    """Build a complete Commit object from a GitPython commit."""

    # Build GitMetadata
    git_metadata = GitMetadata(
        sha=GitSHA(git_commit.hexsha),
        author=create_git_actor(git_commit.author, git_commit.authored_date),
        committer=create_git_actor(git_commit.committer, git_commit.committed_date),
        parents=[GitSHA(p.hexsha) for p in git_commit.parents],
        gpg_signature=(
            git_commit.gpgsig
            if hasattr(git_commit, "gpgsig") and git_commit.gpgsig
            else None
        ),
    )

    # Extract file modifications
    modifications = extract_file_modifications(git_commit)

    # Create affected_paths as list of tuples
    affected_paths = [(mod.path_before, mod.path_after) for mod in modifications]

    # Build Diff object
    diff = Diff(
        modifications=modifications,
        files_changed_count=len(modifications),
        insertions_count=sum(mod.insertions for mod in modifications),
        deletions_count=sum(mod.deletions for mod in modifications),
        affected_paths=affected_paths,
    )

    # Get commit message parts
    commit_lines = git_commit.message.strip().split("\n")
    summary = commit_lines[0] if commit_lines else ""
    full_message = git_commit.message.strip()

    # Get branches and tags
    branches = get_branches_containing_commit(repo, git_commit.hexsha)
    tags = get_tags_for_commit(repo, git_commit.hexsha)

    # Build the complete Commit object
    return Commit(
        metadata=git_metadata,
        summary=summary,
        message=full_message,
        branches=branches,
        tags=tags,
        diff=diff,
        ai_summary=None,
    )

## Build Commit Objects

Process all commits and build structured data objects.

In [6]:
# Build Commit objects for all 5 commits
commit_objects = []
build_errors = []

console.print("\n[bold cyan]Building Commit Objects...[/bold cyan]\n")

for i, git_commit in enumerate(commits_to_process):
    try:
        commit_obj = build_commit_object(git_commit, repo)
        commit_objects.append(commit_obj)
        console.print(
            f"✅ Successfully built commit {i + 1}: {commit_obj.get_short_sha()}"
        )
    except Exception as e:
        build_errors.append((git_commit.hexsha[:8], str(e)))
        console.print(
            f"❌ Failed to build commit {i + 1}: {git_commit.hexsha[:8]} - {e!s}",
            style="red",
        )

num_commits = len(commits_to_process)
num_success = len(commit_objects)
console.print(
    f"\n[bold green]Successfully built {num_success} out of "
    f"{num_commits} commits[/bold green]"
)

if build_errors:
    console.print("\n[bold red]Build Errors:[/bold red]")
    for sha, error in build_errors:
        console.print(f"  {sha}: {error}")

Building Commit Objects...

✅ Successfully built commit 1: c136f0c9

✅ Successfully built commit 2: 2110ee30

✅ Successfully built commit 3: c56337bb

✅ Successfully built commit 4: d75065a7

✅ Successfully built commit 5: c0a82db3

Successfully built 5 out of 5 commits

## Display Commit Information

Show detailed information for each commit in a structured format.

In [7]:
# Display detailed information for each commit
for i, commit_obj in enumerate(commit_objects):
    # Create a tree structure for commit details
    tree = Tree(
        f"[bold magenta]Commit {i + 1}: {commit_obj.get_short_sha()}[/bold magenta]"
    )

    # Metadata branch
    metadata_branch = tree.add("[bold blue]Metadata[/bold blue]")
    metadata_branch.add(f"Full SHA: {commit_obj.metadata.sha}")
    author_email = commit_obj.metadata.author.email
    metadata_branch.add(f"Author: {commit_obj.metadata.author.name} <{author_email}>")
    metadata_branch.add(f"Date: {commit_obj.metadata.author.timestamp}")
    parent_shas = ", ".join([p[:8] for p in commit_obj.metadata.parents])
    metadata_branch.add(f"Parents: {len(commit_obj.metadata.parents)} ({parent_shas})")
    metadata_branch.add(
        f"Signed: {'Yes' if commit_obj.metadata.gpg_signature else 'No'}"
    )

    # Message branch
    message_branch = tree.add("[bold green]Message[/bold green]")
    message_branch.add(f"Summary: {commit_obj.summary}")
    if "\n" in commit_obj.message and len(commit_obj.message.split("\n")) > 1:
        message_branch.add("Full message: [See below]")

    # Repository info branch
    repo_branch = tree.add("[bold yellow]Repository Info[/bold yellow]")
    repo_branch.add(
        f"Branches: {', '.join(commit_obj.branches) if commit_obj.branches else 'None'}"
    )
    repo_branch.add(
        f"Tags: {', '.join(commit_obj.tags) if commit_obj.tags else 'None'}"
    )

    # Diff statistics branch
    diff_branch = tree.add("[bold red]Changes[/bold red]")
    diff_branch.add(f"Files changed: {commit_obj.diff.files_changed_count}")
    diff_branch.add(f"Insertions: +{commit_obj.diff.insertions_count}")
    diff_branch.add(f"Deletions: -{commit_obj.diff.deletions_count}")
    diff_branch.add(f"Total changes: {commit_obj.get_total_changes()}")

    # File modifications
    if commit_obj.diff.modifications:
        files_branch = diff_branch.add("Modified files:")
        for mod in commit_obj.diff.modifications[:5]:  # Show first 5 files
            file_info = f"{mod.modification_type}: "
            if mod.modification_type == "R":
                file_info += f"{mod.path_before} → {mod.path_after}"
            else:
                file_info += mod.path_after or mod.path_before
            file_info += f" (+{mod.insertions}/-{mod.deletions})"
            files_branch.add(file_info)
        if len(commit_obj.diff.modifications) > 5:
            files_branch.add(
                f"... and {len(commit_obj.diff.modifications) - 5} more files"
            )

    console.print(tree)

    # Display full commit message if it's multi-line
    if "\n" in commit_obj.message and len(commit_obj.message.split("\n")) > 1:
        console.print(
            Panel(commit_obj.message, title="Full Commit Message", border_style="green")
        )

    console.print("")  # Add spacing between commits

Commit 1: c136f0c9
├── Metadata
│   ├── Full SHA: c136f0c957da5b56f0f6174d7c78eeddfbf6e6a9
│   ├── Author: 00arun00 <arunjoseph.ai@gmail.com>
│   ├── Date: 2025-07-05 17:19:54
│   ├── Parents: 1 (2110ee30)
│   └── Signed: No
├── Message
│   ├── Summary: feat(dev): add Jupyter notebook support with dependencies
│   └── Full message: [See below]
├── Repository Info
│   ├── Branches: aj/ai-83-jupyter-notebook-support
│   └── Tags: None
└── Changes
    ├── Files changed: 3
    ├── Insertions: +968
    ├── Deletions: -123
    ├── Total changes: 1091
    └── Modified files:
        ├── M: notebooks/build_commit_object.ipynb (+889/-123)
        ├── A: pr-description-updated.md (+28/-0)
        └── A: pr-description.md (+51/-0)

╭────────────────────────────────────────────── Full Commit Message ──────────────────────────────────────────────╮
│ feat(dev): add Jupyter notebook support with dependencies                                                       │
│                                                                                                                 │
│ - Add notebook dependencies to pyproject.toml:                                                                  │
│   - nbqa>=1.7.0,<2.0.0 for quality assurance                                                                    │
│   - ipykernel>=6.0.0,<7.0.0 for Jupyter kernel                                                                  │
│   - jupyter>=1.0.0,<2.0.0 for notebook interface                                                                │
│   - jupyter-ruff>=0.3.0,<1.0.0 for formatting                                                                   │
│ - Create notebooks/ directory structure                                                                         │
│ - Add build_commit_object notebook for testing data models                                                      │
│ - Include commit_objects_export.json for test data                                                              │
│ - Add sandbox.py for quick experiments                                                                          │
│ - Update secrets baseline to allow Git SHAs in test data                                                        │
│ - Fix pre-commit nbQA configuration for newer version                                                           │
│ - Configure ruff to ignore notebook-specific patterns                                                           │
│                                                                                                                 │
│ This improves developer experience for testing and prototyping                                                  │
│ with interactive notebooks.                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Commit 2: 2110ee30
├── Metadata
│   ├── Full SHA: 2110ee3033c74cc5452c2e431229c0f73f5b9c1c
│   ├── Author: 00arun00 <arunjoseph.ai@gmail.com>
│   ├── Date: 2025-07-05 17:19:54
│   ├── Parents: 1 (c56337bb)
│   └── Signed: No
├── Message
│   ├── Summary: feat(dev): add Jupyter notebook support with dependencies
│   └── Full message: [See below]
├── Repository Info
│   ├── Branches: aj/ai-83-jupyter-notebook-support, main
│   └── Tags: None
└── Changes
    ├── Files changed: 7
    ├── Insertions: +2163
    ├── Deletions: -244
    ├── Total changes: 2407
    └── Modified files:
        ├── M: .pre-commit-config.yaml (+4/-2)
        ├── M: .secrets.baseline (+52/-1)
        ├── A: notebooks/build_commit_object.ipynb (+708/-0)
        ├── A: notebooks/commit_objects_export.json (+330/-0)
        ├── A: notebooks/sandbox.py (+5/-0)
        └── ... and 2 more files

╭────────────────────────────────────────────── Full Commit Message ──────────────────────────────────────────────╮
│ feat(dev): add Jupyter notebook support with dependencies                                                       │
│                                                                                                                 │
│ - Add notebook dependencies to pyproject.toml:                                                                  │
│   - nbqa>=1.7.0,<2.0.0 for quality assurance                                                                    │
│   - ipykernel>=6.0.0,<7.0.0 for Jupyter kernel                                                                  │
│   - jupyter>=1.0.0,<2.0.0 for notebook interface                                                                │
│   - jupyter-ruff>=0.3.0,<1.0.0 for formatting                                                                   │
│ - Create notebooks/ directory structure                                                                         │
│ - Add build_commit_object notebook for testing data models                                                      │
│ - Include commit_objects_export.json for test data                                                              │
│ - Add sandbox.py for quick experiments                                                                          │
│ - Update secrets baseline to allow Git SHAs in test data                                                        │
│ - Fix pre-commit nbQA configuration for newer version                                                           │
│ - Configure ruff to ignore notebook-specific patterns                                                           │
│                                                                                                                 │
│ This improves developer experience for testing and prototyping                                                  │
│ with interactive notebooks.                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Commit 3: c56337bb
├── Metadata
│   ├── Full SHA: c56337bb2a05fe3153b472dabd9b5352c87a5a24
│   ├── Author: 00arun00 <arunjoseph.ai@gmail.com>
│   ├── Date: 2025-07-05 16:55:22
│   ├── Parents: 1 (d75065a7)
│   └── Signed: No
├── Message
│   ├── Summary: refactor(tests): migrate to modular strategy architecture
│   └── Full message: [See below]
├── Repository Info
│   ├── Branches: aj/ai-83-jupyter-notebook-support, aj/ai-88-add-codecov-to-pr-checks, main
│   └── Tags: None
└── Changes
    ├── Files changed: 15
    ├── Insertions: +2317
    ├── Deletions: -530
    ├── Total changes: 2847
    └── Modified files:
        ├── M: .secrets.baseline (+5/-5)
        ├── A: tests/data_models/strategies/README.md (+110/-0)
        ├── A: tests/data_models/strategies/__init__.py (+144/-0)
        ├── A: tests/data_models/strategies/actors.py (+198/-0)
        ├── A: tests/data_models/strategies/base.py (+155/-0)
        └── ... and 10 more files

╭────────────────────────────────────────────── Full Commit Message ──────────────────────────────────────────────╮
│ refactor(tests): migrate to modular strategy architecture                                                       │
│                                                                                                                 │
│ - Delete monolithic tests/data_models/test_strategies.py                                                        │
│ - Create modular strategies/ directory with domain-specific modules:                                            │
│   - base.py: Common utilities and helper functions                                                              │
│   - utils.py: GitSHA and GPG signature strategies                                                               │
│   - actors.py: GitActor-specific strategies                                                                     │
│   - metadata.py: GitMetadata and ChangeMetadata strategies                                                      │
│   - files.py: FileModification and Diff strategies                                                              │
│   - commits.py: Commit-specific strategies                                                                      │
│ - Update all test files to use new modular imports                                                              │
│ - Strategies now generate model instances instead of raw data                                                   │
│ - Improve type safety and composability                                                                         │
│                                                                                                                 │
│ This refactoring improves test maintainability by organizing strategies                                         │
│ by domain and providing better building blocks for complex test scenarios.                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Commit 4: d75065a7
├── Metadata
│   ├── Full SHA: d75065a782f907bde069da805814873bed82d450
│   ├── Author: 00arun00 <arunjoseph.ai@gmail.com>
│   ├── Date: 2025-07-05 17:21:50
│   ├── Parents: 1 (c0a82db3)
│   └── Signed: No
├── Message
│   ├── Summary: docs: add comprehensive testing patterns documentation
│   └── Full message: [See below]
├── Repository Info
│   ├── Branches: aj/ai-83-jupyter-notebook-support, aj/ai-88-add-codecov-to-pr-checks, main
│   └── Tags: None
└── Changes
    ├── Files changed: 2
    ├── Insertions: +1078
    ├── Deletions: -424
    ├── Total changes: 1502
    └── Modified files:
        ├── A: docs/test-strategy-refactoring.md (+559/-0)
        └── M: docs/testing-patterns.md (+519/-424)

╭────────────────────────────────────────────── Full Commit Message ──────────────────────────────────────────────╮
│ docs: add comprehensive testing patterns documentation                                                          │
│                                                                                                                 │
│ - Update docs/testing-patterns.md with:                                                                         │
│   - Overview of modular strategy architecture                                                                   │
│   - Examples of strategy composition                                                                            │
│   - Best practices for creating new strategies                                                                  │
│   - Migration guide from old patterns                                                                           │
│   - Detailed examples for each strategy type                                                                    │
│ - Add docs/test-strategy-refactoring.md outlining:                                                              │
│   - Potential future improvements                                                                               │
│   - Co-location of strategies with data models                                                                  │
│   - Benefits of the refactoring approach                                                                        │
│                                                                                                                 │
│ This documentation helps developers understand and effectively use                                              │
│ the new modular testing infrastructure.                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Commit 5: c0a82db3
├── Metadata
│   ├── Full SHA: c0a82db39e99d00c5ee920901c03b038f4fc7ab2
│   ├── Author: dependabot <49699333+dependabot@users.noreply.github.com>
│   ├── Date: 2025-07-06 05:21:10
│   ├── Parents: 1 (c71479b8)
│   └── Signed: No
├── Message
│   ├── Summary: ci(deps): bump the actions group with 6 updates
│   └── Full message: [See below]
├── Repository Info
│   ├── Branches: aj/ai-83-jupyter-notebook-support, aj/ai-88-add-codecov-to-pr-checks, main, 
│   │   test-refactoring-with-docs, test-refactoring-with-docs-backup
│   └── Tags: None
└── Changes
    ├── Files changed: 12
    ├── Insertions: +49
    ├── Deletions: -49
    ├── Total changes: 98
    └── Modified files:
        ├── M: .github/workflows/ci.yml (+1/-1)
        ├── M: .github/workflows/developer-tools.yml (+7/-7)
        ├── M: .github/workflows/main-branch.yml (+7/-7)
        ├── M: .github/workflows/nightly.yml (+5/-5)
        ├── M: .github/workflows/pr-checks.yml (+4/-4)
        └── ... and 7 more files

╭────────────────────────────────────────────── Full Commit Message ──────────────────────────────────────────────╮
│ ci(deps): bump the actions group with 6 updates                                                                 │
│                                                                                                                 │
│ Bumps the actions group with 6 updates:                                                                         │
│                                                                                                                 │
│ | Package | From | To |                                                                                         │
│ | --- | --- | --- |                                                                                             │
│ | (https://github.com/actions/checkout) | `4.1.7` | `4.2.2` |                                                   │
│ | (https://github.com/peter-evans/create-pull-request) | `7.0.5` | `7.0.8` |                                    │
│ | (https://github.com/actions/upload-artifact) | `4.4.0` | `4.6.2` |                                            │
│ | (https://github.com/github/codeql-action) | `3.26.6` | `3.29.2` |                                             │
│ | (https://github.com/actions/download-artifact) | `4.1.8` | `4.3.0` |                                          │
│ | (https://github.com/softprops/action-gh-release) | `2.0.8` | `2.3.2` |                                        │
│                                                                                                                 │
│                                                                                                                 │
│ Updates `actions/checkout` from 4.1.7 to 4.2.2                                                                  │
│ - [Release notes](https://github.com/actions/checkout/releases)                                                 │
│ - [Changelog](https://github.com/actions/checkout/blob/main/CHANGELOG.md)                                       │
│ -                                                                                                               │
│ [Commits](https://github.com/actions/checkout/compare/692973e3d937129bcbf40652eb9f2f61becf3332...11bd71901bbe5b │
│ 1630ceea73d27597364c9af683)                                                                                     │
│                                                                                                                 │
│ Updates `peter-evans/create-pull-request` from 7.0.5 to 7.0.8                                                   │
│ - [Release notes](https://github.com/peter-evans/create-pull-request/releases)                                  │
│ -                                                                                                               │
│ [Commits](https://github.com/peter-evans/create-pull-request/compare/5e914681df9dc83aa4e4905692ca88beb2f9e91f.. │
│ .271a8d0340265f705b14b6d32b9829c1cb33d45e)                                                                      │
│                                                                                                                 │
│ Updates `actions/upload-artifact` from 4.4.0 to 4.6.2                                                           │
│ - [Release notes](https://github.com/actions/upload-artifact/releases)                                          │
│ -                                                                                                               │
│ [Commits](https://github.com/actions/upload-artifact/compare/50769540e7f4bd5e21e526ee35c689e35e0d6874...ea165f8 │
│ d65b6e75b540449e92b4886f43607fa02)                                                                              │
│                                                                                                                 │
│ Updates `github/codeql-action` from 3.26.6 to 3.29.2  

## Aggregate Statistics

Show summary statistics across all processed commits.

In [8]:
# Aggregate statistics across all commits
if commit_objects:
    console.print("[bold cyan]Aggregate Statistics[/bold cyan]\n")

    # Calculate totals
    total_files_changed = sum(c.diff.files_changed_count for c in commit_objects)
    total_insertions = sum(c.diff.insertions_count for c in commit_objects)
    total_deletions = sum(c.diff.deletions_count for c in commit_objects)
    total_changes = sum(c.get_total_changes() for c in commit_objects)

    # File modification type distribution
    mod_type_counts = defaultdict(int)
    for commit in commit_objects:
        for mod in commit.diff.modifications:
            mod_type_counts[mod.modification_type] += 1

    # Create statistics table
    stats_table = Table(
        title="Summary Statistics", show_header=True, header_style="bold cyan"
    )
    stats_table.add_column("Metric", style="white")
    stats_table.add_column("Value", style="yellow")

    stats_table.add_row("Total Commits", str(len(commit_objects)))
    stats_table.add_row("Total Files Changed", str(total_files_changed))
    stats_table.add_row("Total Insertions", f"+{total_insertions}")
    stats_table.add_row("Total Deletions", f"-{total_deletions}")
    stats_table.add_row("Total Line Changes", str(total_changes))
    stats_table.add_row(
        "Average Changes per Commit", f"{total_changes / len(commit_objects):.1f}"
    )

    console.print(stats_table)

    # Modification type distribution
    if mod_type_counts:
        mod_table = Table(
            title="File Modification Types", show_header=True, header_style="bold cyan"
        )
        mod_table.add_column("Type", style="white")
        mod_table.add_column("Description", style="green")
        mod_table.add_column("Count", style="yellow")

        mod_type_descriptions = {
            "A": "Added",
            "M": "Modified",
            "D": "Deleted",
            "R": "Renamed",
            "C": "Copied",
            "T": "Type changed",
            "U": "Unmerged",
            "X": "Unknown",
            "B": "Broken pairing",
        }

        for mod_type, count in sorted(
            mod_type_counts.items(), key=lambda x: x[1], reverse=True
        ):
            mod_table.add_row(
                mod_type, mod_type_descriptions.get(mod_type, "Unknown"), str(count)
            )

        console.print("\n")
        console.print(mod_table)

Aggregate Statistics

          Summary Statistics           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric                     ┃ Value  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Total Commits              │ 5      │
│ Total Files Changed        │ 39     │
│ Total Insertions           │ +6575  │
│ Total Deletions            │ -1370  │
│ Total Line Changes         │ 7945   │
│ Average Changes per Commit │ 1589.0 │
└────────────────────────────┴────────┘

   File Modification Types    
┏━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━┓
┃ Type ┃ Description ┃ Count ┃
┡━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━┩
│ M    │ Modified    │ 24    │
│ A    │ Added       │ 14    │
│ D    │ Deleted     │ 1     │
└──────┴─────────────┴───────┘

## Export to JSON

Export the commit objects to JSON format for further processing or storage.

In [9]:
# Export commits to JSON for inspection
if commit_objects:
    console.print("\n[bold cyan]JSON Export[/bold cyan]\n")

    # Export the first commit as formatted JSON
    first_commit_json = commit_objects[0].model_dump_json(indent=2)

    # Show a preview of the JSON structure
    json_preview = (
        first_commit_json[:800] + "..."
        if len(first_commit_json) > 800
        else first_commit_json
    )

    syntax = Syntax(json_preview, "json", theme="monokai", line_numbers=True)
    console.print(
        Panel(
            syntax,
            title=f"JSON Export Preview: {commit_objects[0].get_short_sha()}",
            border_style="cyan",
        )
    )

    # Save all commits to a JSON file
    all_commits_data = [commit.model_dump() for commit in commit_objects]
    output_file = project_root / "notebooks" / "commit_objects_export.json"

    with open(output_file, "w") as f:
        json.dump(all_commits_data, f, indent=2, default=str)

    num_commits = len(commit_objects)
    console.print(
        f"\n[green]✅ Exported all {num_commits} commits to:[/green] {output_file}"
    )

JSON Export

╭───────────────────────────────────────── JSON Export Preview: c136f0c9 ─────────────────────────────────────────╮
│    1 {                                                                                                          │
│    2   "metadata": {                                                                                            │
│    3     "sha": "c136f0c957da5b56f0f6174d7c78eeddfbf6e6a9",                                                     │
│    4     "author": {                                                                                            │
│    5       "name": "00arun00",                                                                                  │
│    6       "email": "arunjoseph.ai@gmail.com",                                                                  │
│    7       "timestamp": "2025-07-05T17:19:54"                                                                   │
│    8     },                                                                                                     │
│    9     "committer": {                                                                                         │
│   10       "name": "00arun00",                                                                                  │
│   11       "email": "arunjoseph.ai@gmail.com",                                                                  │
│   12       "timestamp": "2025-07-06T10:48:46"                                                                   │
│   13     },                                                                                                     │
│   14     "parents": [                                                                                           │
│   15       "2110ee3033c74cc5452c2e431229c0f73f5b9c1c"                                                           │
│   16     ],                                                                                                     │
│   17     "gpg_signature": null                                                                                  │
│   18   },                                                                                                       │
│   19   "summary": "feat(dev): add Jupyter notebook support with dependencies",                                  │
│   20   "message": "feat(dev): add Jupyter notebook support with dependencies\n\n- Add notebook dependencies to  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Exported all 5 commits to: /home/aj/code/auto-release-note-generation/notebooks/commit_objects_export.json